In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import OneHotEncoder
import random
sns.set()

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
train['title'] = train.Name.apply(lambda x: x.split(',')[1].split('.')[0])

In [5]:
test['title'] = test.Name.apply(lambda x: x.split(',')[1].split('.')[0])

In [6]:
train['Age'] = train.Age.fillna(train.Age.mean())
test['Age'] = test.Age.fillna(test.Age.mean())
train['Embarked'] = train.Embarked.fillna('S')
test['Embarked'] = test.Embarked.fillna('S')

In [8]:
train['Cabin_letter'] = train.Cabin.astype(str).apply(lambda x: re.sub(r'^(.).*', r'\1',x))

In [9]:
counts = train.Cabin_letter.value_counts()
distribution = counts / counts.sum()
distribution = pd.DataFrame(distribution.drop(labels=['n']))
distribution.rename(columns={'Cabin_letter':'prob'}, inplace=True)

In [10]:
train['Cabin_letter'] = train.Cabin_letter.apply(lambda x: random.choices(distribution.index, weights=distribution.prob)[0] if x =='n' else x)

In [29]:
cat_df = pd.DataFrame([train.Pclass,train.Sex,train.Embarked,train.Cabin_letter]).T

In [39]:
num_df = pd.DataFrame([train.Age,train.SibSp,train.Parch,train.Fare]).T

,Age,SibSp,Parch,Fare
0,22.000000,1.0,0.0,7.2500
1,38.000000,1.0,0.0,71.2833
2,26.000000,0.0,0.0,7.9250
3,35.000000,1.0,0.0,53.1000
4,35.000000,0.0,0.0,8.0500
...,...,...,...,...
886,27.000000,0.0,0.0,13.0000
887,19.000000,0.0,0.0,30.0000
888,29.699118,1.0,2.0,23.4500
889,26.000000,0.0,0.0,30.0000


In [30]:
ohe = OneHotEncoder()
cat_df = ohe.fit_transform(cat_df).toarray()

In [37]:
cat_df = pd.DataFrame(cat_df,columns=ohe.get_feature_names_out())
cat_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
887,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [43]:
final_df = pd.concat([cat_df,num_df,train.Survived],axis=1)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,y_train,y_test = train_test_split(final_df.iloc[:,:-1],final_df.iloc[:,-1], test_size=0.3, shuffle=True)

In [47]:
rf = RandomForestClassifier()

rf.fit(X_train,y_train)

rf.score(X_test,y_test)

0.8134328358208955

In [64]:
from sklearn.model_selection import cross_validate

scoring = ['accuracy','recall','precision','f1']

cross_v = pd.DataFrame(cross_validate(rf, final_df.iloc[:,:-1],final_df.iloc[:,-1], cv=10, scoring = scoring))

In [65]:
for score in scoring:
    print(f'{score} is: ', cross_v['test_'+str(score)].mean())

accuracy is:  0.8093008739076154
recall is:  0.7164705882352942
precision is:  0.7725683028293773
f1 is:  0.7394339362940758


In [66]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title,Cabin_letter
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S,Mr,E
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C85,C,Mrs,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,B
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,C123,S,Mrs,C
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S,Mr,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,NaN,S,Rev,D
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,B42,S,Miss,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,NaN,S,Miss,B
889,890,1,1,"Behr, Mr. Karl Howell",male,26.000000,0,0,111369,30.0000,C148,C,Mr,C


In [88]:
train[['Survived','Cabin_letter']].groupby('Cabin_letter',as_index=False).value_counts()

,Cabin_letter,Survived,count
0,A,0,48
1,A,1,24
2,B,0,130
3,B,1,86
4,C,0,171
5,C,1,90
6,D,0,75
7,D,1,59
8,E,0,80
9,E,1,57
